In [1]:
# ! git clone https://github.com/ClaudioPaterniti/MinesweeperLearning.git

In [2]:
# ! pip install plotly-express

In [3]:
import os
# os.chdir('/content/MinesweeperLearning')

In [4]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from src.models.patch_mlp import MineSweeperDataset, PatchMLPModel, Game
from src.player import ThresholdPlayer

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(device)

cpu


In [6]:
name = f'patch_mlp_3x3_128_halving'
patch_radius = 1
model = PatchMLPModel(
    patch_radius, layers=[2**i for i in range(7, 0, -1)], mine_rate_channel=True, device=device)
player = ThresholdPlayer(model, 0.05, 0.95)
optimizer = torch.optim.Adam(model.model.parameters(), lr=0.001)

In [7]:
train = Game(16, 30, 99, n = 1000)
train.random_open(0.3)
train.random_flags(0.3)
test = Game(n = 100)
test.random_open(0.3)
test.random_flags(0.3)

In [8]:
transform = model.transform
training_data = MineSweeperDataset(train, transform, losing_moves_weight=11)
test_data = MineSweeperDataset(test, transform)
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    model.train(train_dataloader, optimizer)
    model.test(test_dataloader)

Epoch 1
-------------------------------
Epoch 2
-------------------------------
Epoch 3
-------------------------------
Epoch 4
-------------------------------
Epoch 5
-------------------------------


In [11]:
reinforcing_iterations = 10
epochs = 2
for i in range(reinforcing_iterations):
    games = Game(16, 30, 99, n = 1000)
    games.random_open(0.3)
    player.play(games)
    print(f'average score: {np.average(games.scores())}')
    training_data.mix(games)
    train_dataloader = DataLoader(training_data, batch_size=64)
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        model.train(train_dataloader, optimizer)
        model.test(test_dataloader)

average score: -1.559436025980245
Epoch 1
-------------------------------
Epoch 2
-------------------------------
average score: -2.416201382340479
Epoch 1
-------------------------------
Epoch 2
-------------------------------
average score: -2.769893977911453
Epoch 1
-------------------------------
Epoch 2
-------------------------------
average score: -3.9588677746354444
Epoch 1
-------------------------------
Epoch 2
-------------------------------
average score: -4.052556703665582
Epoch 1
-------------------------------
Epoch 2
-------------------------------
average score: -4.494741059598824
Epoch 1
-------------------------------
Epoch 2
-------------------------------
average score: -4.50539305415116
Epoch 1
-------------------------------
Epoch 2
-------------------------------
average score: -4.502265239185517
Epoch 1
-------------------------------
Epoch 2
-------------------------------
average score: -4.504160875111278
Epoch 1
-------------------------------
Epoch 2
------

In [12]:
model.save(os.path.join('weights', name + '.pth'))

In [13]:
import plotly_express as px
import pandas as pd
df = pd.DataFrame({'train': model.train_loss_log, 'test': model.test_loss_log})
px.line(df)